<a href="https://colab.research.google.com/github/somilasthana/deeplearningtutorial/blob/master/Making_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np


In [0]:
"""
Theory:

CNN applies series of filter on raw pixel data of an image to extract and learn high-level features.

1. Convolutional layers:
2. Pooling layers
3. Dense (fully connected) layers

Convolution layers
For each subregion, the layer performs a set of mathematical operations to produce a single value in the output feature map. 

Pooling layers
Downsamples or reduces the dimensionality of the feature map in order to decrease processing time.

Dense (fully connected) layers
Performs the classification on the convoluted downsampled layer

"""

CNN will be build with following architecture



*   Convolutional Layer #1 (CL#1) : 32 , 5X5 filters
*   Pooling Layer #1 (PL#1) : Max Pooling 2X2 filters stride=2
*   CL#2 : 64, 5X5 filters
*   PL#2 : Max Pooling 2X2 filter stride=2
*   Dense Layer #1(DL#1) : 1,024 neurons , dropout(0.4)
*   DL#2 : softmax, 10 neurons



In [0]:
def cnn_model_fn(features, labels, mode):
  # Input Layer
  """Each input tensor expects a shape of [batch_size, image_height, image_width, channels]
     The reason for batch_size = -1 so that it can be tuned as a hyperparameter.
  """
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
  
  """CL# 1 [batch_size, 28, 28, 1] ===becomes===> [batch_size, 28, 28, 32]"""
  
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu
  )
  
  """PL#1 2x2 filter reduces height and width by 50% each."""
  """[batch_size, 28, 28, 32] ===becomes===> [batch_size, 14, 14, 32]"""
  
  pool1 = tf.layers.max_pooling2d(
      inputs=conv1,
      pool_size=[2 ,2],
      strides=2
  )
  
  """CL# 2 [batch_size, 14, 14, 32] ===becomes===> [batch_size, 14, 14, 64]"""
  
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu
  )
  
  """PL#2 [batch_size, 14, 14, 64] ===becomes===> [batch_size, 7, 7, 64]"""
  
  
  pool2 = tf.layers.max_pooling2d(
      inputs=conv2,
      pool_size=[2,2],
      strides=2
  )
  
  """Flatten to feed it to dense layers"""
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  
  """DL# 1 [batch_size, 3136] ---feeds to ---> 1024 neurons"""
  """ shape is [batch_size, 1024] """
  
  dense = tf.layers.dense(
      inputs=pool2_flat,
      units=1024,
      activation=tf.nn.relu
  )
  
  # Regularization
  """ No shape change [batch_size, 1024] """
  dropout = tf.layers.dropout(
      inputs=dense,
      rate=0.4,
      training= mode == tf.estimator.ModeKeys.TRAIN
  )
  
  """The output is [batch_size, 10]"""
  logits = tf.layers.dense(inputs=dropout, units=10)
  
  predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimateSpecs(mode=mode, predictions=predictions)
  
  # Calculate Loss
  
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  
  # Configure the Training Op (for TRAIN mode)
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizier = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    
    train_op = optimizier.minimize(
        loss=loss,
        global_step=tf.train.get_global_step()
    )
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
  
  
  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels,
          predictions=predictions["classes"]
      )
  }
  
  return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
  
  
  
  
  
  
  
  
  

Load Training and Test Dataset

In [0]:
# Load training and eval data
((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

In [47]:
# Create the Estimator
# a TensorFlow class for performing high-level model training, evaluation, and inference) for our model
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,
    model_dir="/tmp/mnist_convnet_model"
)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fba6a6c5f60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x": train_data},
    y = train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True
)

In [49]:
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 2.2962666, step = 1
INFO:tensorflow:Saving checkpoints for 2 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 2.2962666.


In [50]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-03T10:30:26Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-2
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-03-10:30:26
INFO:tensorflow:Saving dict for global step 2: accuracy = 0.1354, global_step = 2, loss = 2.2978826
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2: /tmp/mnist_convnet_model/model.ckpt-2
{'accuracy': 0.1354, 'loss': 2.2978826, 'global_step': 2}
